# Training Data Debug

In [3]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy import spatial
from matplotlib import cm

from tqdm.auto import tqdm

from sphere import Sphere
from dh_grid import DHGrid
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Load the previous (old) data format

In [78]:
export_ds = '/media/berlukas/Data/data/nuscenes/'

cloud_filename = f"{export_ds}/sem_clouds_decoded.npy"
cloud_features_decoded = np.load(cloud_filename)
cloud_features_decoded = np.argmax(cloud_features_decoded, axis=1)

print(f"Shape of clouds is {cloud_features_decoded.shape}")

Shape of clouds is (100, 200, 200)


# Load the current data format

In [96]:
export_ds = '/media/berlukas/Data/data/nuscenes/'

# cloud_filename = f"{export_ds}/sem_clouds_08.npy"
cloud_filename = f"{export_ds}/sem_clouds_val_kitti_final.npy"
# cloud_filename = f"{export_ds}/sem_clouds_decoded.npy"

print(f"Loading clouds from {cloud_filename}")
cloud_features = np.load(cloud_filename)
print(f"Initial shape of clouds is {cloud_features.shape}")
sem_cloud_features = cloud_features[:, 2, :, :]
cloud_features = cloud_features[:, 0:2, :, :]

print(f"Shape of clouds is {cloud_features.shape}")
# print(f"Shape of clouds is {cloud_features.shape} and sem clouds is {sem_cloud_features.shape}")

Loading clouds from /media/berlukas/Data/data/nuscenes//sem_clouds_val_kitti_final.npy
Initial shape of clouds is (100, 3, 200, 200)
Shape of clouds is (100, 2, 200, 200)


## Compare the two datasets

In [93]:
def mapIntensityToRGB(i):
    mask = np.where(i < 0)    
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    colors[mask] = 0
    return colors

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)        
    o3d.visualization.draw_geometries([pcd])
    
def visualize_sphere(cloud, ndim = 1, bw = 100, idx = 0):
    _, points = DHGrid.CreateGrid(bw)
    cur_sem_cloud = np.reshape(cloud, (ndim, -1)).T
    cur_sem_cloud[:,idx] = cur_sem_cloud[:,idx] * 200
    cur_sem_cloud[cur_sem_cloud[:,idx] < 0] = -1
    
    points_xyz = DHGrid.ConvertPointsToEuclidean(points)                    
    points_xyzi = np.column_stack((points_xyz, cur_sem_cloud[:,idx]))    
    visualize_pointcloud(points_xyzi)
    return points_xyzi
    
def writeRawPointCloud(cloud, filename):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    colors = mapIntensityToRGB(cloud[:, 3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.io.write_point_cloud(filename, pcd)
    

In [79]:
visualize_sphere(cloud_features_decoded[15, :, :], 1, 100)

array([[ 7.85390089e-03,  0.00000000e+00,  9.99969158e-01,
         1.00000000e+00],
       [ 7.85002546e-03,  2.46696989e-04,  9.99969158e-01,
         1.00000000e+00],
       [ 7.83840301e-03,  4.93150517e-04,  9.99969158e-01,
         1.00000000e+00],
       ...,
       [ 7.81904500e-03, -7.39117366e-04, -9.99969158e-01,
         1.00000000e+00],
       [ 7.83840301e-03, -4.93150517e-04, -9.99969158e-01,
         1.00000000e+00],
       [ 7.85002546e-03, -2.46696989e-04, -9.99969158e-01,
         1.00000000e+00]])

In [97]:
xyzi = visualize_sphere(cloud_features[15,:,:], 2, 100, 1)

In [98]:
test = np.unique(xyzi[:, 3])
print(test)

[ -1.           0.           1.99999996   3.99999991   5.99999987
   7.99999982  10.00000015  11.99999973  14.00000006  15.99999964
  18.00000072  20.0000003   21.99999988  23.99999946  25.99999905
  28.00000012  30.00000119  31.99999928  34.00000036  36.00000143
  37.99999952  40.0000006   41.99999869  43.99999976  46.00000083
  47.99999893  50.          51.99999809  54.00000215  56.00000024
  57.99999833  60.00000238  62.00000048  63.99999857  66.00000262
  68.00000072  69.99999881  72.00000286  74.00000095  75.99999905
  77.99999714  80.00000119  81.99999928  83.99999738  86.00000143
  87.99999952  89.99999762  92.00000167  93.99999976  95.99999785
  98.00000191 100.         101.99999809 103.99999619 105.99999428
 108.00000429 110.00000238 112.00000048 113.99999857 115.99999666
 117.99999475 120.00000477 122.00000286 124.00000095 125.99999905
 127.99999714 129.99999523 132.00000525 134.00000334 136.00000143
 137.99999952 139.99999762 144.00000572 146.00000381 148.00000191
 150.     

In [63]:
[ -1.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.
  13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.
  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.
  41.  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.
  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.
  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.
  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  96.  97.
  98.  99. 100. 101. 102. 103. 106. 107. 109. 120. 174.]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1.])